# [E-15] 아이유팬이 좋아할 만한 다른 아티스트 찾기

### 목차


1. 라이브러리 Import 
2. 데이터 준비 & 전처리
3. 분석해 봅시다.
4. 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.
5. CSR matrix를 직접 만들어 봅시다.
6. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.
7. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.
8. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.
9. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.


### 목표 

<br> 

|평가문항|상세기준|
|:---|:---|
| 1. CSR matrix가 정상적으로 만들어졌다.| 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.|
|2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.|사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.|
|3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.| MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.|

<br>

### (1) 라이브러리 import

In [1]:
import numpy as np
import pandas as pd
import os

import scipy
import implicit
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

### (2) 데이터 준비 & 전처리

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
# 사용할 컬럼
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# Data filtering 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)
ratings

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
movies.tail()

,movie_id,title,genre
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


### (3) 분석해 봅시다.

- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [7]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [8]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [9]:
# 가장 인기 있는 영화 30개(인기순)
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_top30 = movie_count.sort_values(ascending=False).head(30)

In [10]:
# 30개의 영화 누적 count수
for i, k in zip(movie_top30.index, movie_top30.values):
    print(movies[movies['movie_id']==i]['title'].values[0],k)

American Beauty (1999) 3211
Star Wars: Episode IV - A New Hope (1977) 2910
Star Wars: Episode V - The Empire Strikes Back (1980) 2885
Star Wars: Episode VI - Return of the Jedi (1983) 2716
Saving Private Ryan (1998) 2561
Terminator 2: Judgment Day (1991) 2509
Silence of the Lambs, The (1991) 2498
Raiders of the Lost Ark (1981) 2473
Back to the Future (1985) 2460
Matrix, The (1999) 2434
Jurassic Park (1993) 2413
Sixth Sense, The (1999) 2385
Fargo (1996) 2371
Braveheart (1995) 2314
Men in Black (1997) 2297
Schindler's List (1993) 2257
Princess Bride, The (1987) 2252
Shakespeare in Love (1998) 2213
L.A. Confidential (1997) 2210
Shawshank Redemption, The (1994) 2194
Godfather, The (1972) 2167
Groundhog Day (1993) 2121
E.T. the Extra-Terrestrial (1982) 2102
Being John Malkovich (1999) 2066
Ghostbusters (1984) 2051
Pulp Fiction (1994) 2030
Forrest Gump (1994) 2022
Terminator, The (1984) 2019
Toy Story (1995) 2000
Fugitive, The (1993) 1941


In [11]:
# 유저별 영화를 몇번 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [12]:
# 유저별 count횟수 중앙값에 대한 통계
user_median = ratings.groupby('user_id')['counts'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: counts, dtype: float64

### (4) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [13]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 

my_favorite_name = ['Godfather, The (1972)','Toy Story (1995)','Jurassic Park (1993)','Braveheart (1995)','Shakespeare in Love (1998)']
my_favorite = [movies[movies['title'] == name]['movie_id'].values[0] for name in my_favorite_name]

# jyp(맨끝에 추가되는 id)이라는 user_id가 위 영화를 5회씩 봤다고 가정하겠습니다.
jyp = max(ratings['user_id']) + 1
my_playlist = pd.DataFrame({'user_id': [jyp]*5, 'movie_id': my_favorite, 'counts':[5]*5})

if not ratings.isin({'user_id':[jyp]})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)                 # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 
ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,6041,858,5,NaN
1,6041,1,5,NaN
2,6041,480,5,NaN
3,6041,110,5,NaN
4,6041,2396,5,NaN


In [14]:
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760.0
1,1,661,3,978302109.0
2,1,914,3,978301968.0
3,1,3408,4,978300275.0
4,1,2355,5,978824291.0


### (5) CSR matrix를 직접 만들어 봅시다.

In [15]:
# 실습 위에 설명보고 이해해서 만들어보기

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['counts'], (ratings.user_id, ratings.movie_id)))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### (6) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [16]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [17]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [18]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [19]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### (7) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [20]:
# 영화의 이름을 movie id로 바꿔주는 함수
def movie_name_to_id(name):
    return movies[movies['title']==name]['movie_id'].values[0]

In [21]:
my_vector, braveheart_vector = als_model.user_factors[6041], als_model.item_factors[movie_name_to_id('Braveheart (1995)')]

In [22]:
# 나의 벡터
my_vector

array([ 0.18362272,  0.71919525,  0.88010633,  0.37770355,  0.35041586,
       -0.01760266,  0.19255184,  1.2648852 ,  0.07170111, -0.15745963,
       -0.6001481 , -0.6889914 , -0.45533696,  0.698787  ,  0.45574895,
       -0.11348509,  0.5427132 ,  0.6788932 , -0.38285357, -0.18065137,
        0.03013318,  0.01011967,  0.71139437,  0.49023768, -0.1233808 ,
       -0.52715707, -0.13601334, -0.07407475, -0.39211497, -0.4513541 ,
       -0.00262596,  0.84975344, -0.7773483 , -0.35540462, -0.46521527,
       -0.76043993, -0.5241687 ,  1.2168705 , -0.05560491,  0.52853584,
        0.38879064, -0.6542066 ,  0.11133049,  0.17900462,  0.0498695 ,
       -0.29193428, -0.43320826,  0.4725186 , -0.7858679 , -0.26883236,
        0.8033665 ,  0.66465694,  0.01356362, -0.38192558, -0.11993349,
        0.54138917,  0.35937342, -0.73470116, -0.44599196, -0.11863131,
        0.45036456, -0.5849576 ,  0.2637718 ,  0.31661585,  0.26305282,
       -0.15779781, -0.34162092, -0.22101809, -0.73628956,  0.00

In [23]:
# 브레이브하트의 vector
braveheart_vector 

array([ 2.3918990e-02,  1.9918995e-02, -2.9937556e-02,  1.1442074e-02,
        1.2018979e-02,  5.5637690e-03,  7.5488575e-03,  2.7611108e-02,
        3.6009159e-02, -4.3686554e-03, -3.8630579e-02,  6.4635472e-03,
       -2.4550511e-02,  2.4759108e-02,  1.0139480e-02,  1.5834879e-02,
       -2.4042796e-02,  4.3207772e-02, -4.4050883e-03, -1.2810876e-02,
        1.9384857e-02, -2.7148915e-02,  5.6475163e-03,  1.2837477e-02,
        8.5672429e-03, -1.3381398e-02,  3.9438386e-03,  2.0984707e-03,
        2.2410348e-02,  1.6701362e-03, -1.4576701e-02,  2.1249786e-02,
        2.3195376e-03, -2.8802831e-02, -9.9504797e-04, -1.3594512e-02,
       -3.8310049e-05,  4.9550619e-02, -5.2059470e-03,  3.5670932e-02,
        2.2514261e-02, -1.8343391e-02,  3.4447625e-02,  1.9662390e-02,
        1.8273041e-02,  4.5830183e-02,  5.2752234e-03,  3.9275065e-02,
       -2.8198462e-03, -2.5751747e-02,  2.1365875e-02,  2.3386410e-02,
        5.8826292e-03,  1.1517048e-03,  8.0736224e-03,  6.3135545e-03,
      

In [24]:
# 영화 이름을 넣으면 추천정도를 알려주는 함수
def score_movie(movie_name):
    my_vector, movie_vector = als_model.user_factors[6041], als_model.item_factors[movie_name_to_id(movie_name)]
    return np.dot(my_vector, movie_vector)

In [25]:
for i in movie_top30.index:
    tmp = movies[movies['movie_id']==i]['title'].values[0]
    print(tmp," :",  score_movie(tmp))

American Beauty (1999)  : 0.48415896
Star Wars: Episode IV - A New Hope (1977)  : 0.2015116
Star Wars: Episode V - The Empire Strikes Back (1980)  : 0.1297508
Star Wars: Episode VI - Return of the Jedi (1983)  : 0.12689109
Saving Private Ryan (1998)  : 0.44037175
Terminator 2: Judgment Day (1991)  : 0.42966607
Silence of the Lambs, The (1991)  : 0.039793663
Raiders of the Lost Ark (1981)  : 0.1752045
Back to the Future (1985)  : 0.21992731
Matrix, The (1999)  : 0.25429612
Jurassic Park (1993)  : 0.5250953
Sixth Sense, The (1999)  : 0.07346399
Fargo (1996)  : 0.07889685
Braveheart (1995)  : 0.66898966
Men in Black (1997)  : 0.37167332
Schindler's List (1993)  : 0.12415741
Princess Bride, The (1987)  : 0.10162842
Shakespeare in Love (1998)  : 0.5784813
L.A. Confidential (1997)  : 0.053043872
Shawshank Redemption, The (1994)  : 0.15017885
Godfather, The (1972)  : 0.66203165
Groundhog Day (1993)  : 0.37348035
E.T. the Extra-Terrestrial (1982)  : 0.18214084
Being John Malkovich (1999)  : 0.

### (8) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [26]:
# braveheart 와 비슷한 영화 추천
favorite_movie = 'Braveheart (1995)'
movie_id = movie_name_to_id(favorite_movie)
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(110, 0.99999994),
 (2028, 0.6753715),
 (589, 0.4869167),
 (480, 0.4613618),
 (1610, 0.3700254),
 (2858, 0.368671),
 (457, 0.36539423),
 (527, 0.34831727),
 (2571, 0.34660453),
 (1580, 0.34527937),
 (2887, 0.34229183),
 (989, 0.3321105),
 (266, 0.3258108),
 (1673, 0.3239434),
 (144, 0.32307002)]

In [27]:
for i, k in similar_movie:
    print(movies[movies['movie_id']==i]['title'].values[0],' :', k)

Braveheart (1995)  : 0.99999994
Saving Private Ryan (1998)  : 0.6753715
Terminator 2: Judgment Day (1991)  : 0.4869167
Jurassic Park (1993)  : 0.4613618
Hunt for Red October, The (1990)  : 0.3700254
American Beauty (1999)  : 0.368671
Fugitive, The (1993)  : 0.36539423
Schindler's List (1993)  : 0.34831727
Matrix, The (1999)  : 0.34660453
Men in Black (1997)  : 0.34527937
Simon Sez (1999)  : 0.34229183
Schlafes Bruder (Brother of Sleep) (1995)  : 0.3321105
Legends of the Fall (1994)  : 0.3258108
Boogie Nights (1997)  : 0.3239434
Brothers McMullen, The (1995)  : 0.32307002


In [28]:
#비슷한 영화 장르확인 

movies[movies['movie_id'].isin([s[0] for s in similar_movie])][['title','genre']]

,title,genre
108,Braveheart (1995),Action|Drama|War
142,"Brothers McMullen, The (1995)",Comedy
263,Legends of the Fall (1994),Drama|Romance|War|Western
453,"Fugitive, The (1993)",Action|Thriller
476,Jurassic Park (1993),Action|Adventure|Sci-Fi
523,Schindler's List (1993),Drama|War
585,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
977,Schlafes Bruder (Brother of Sleep) (1995),Drama
1539,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi
1568,"Hunt for Red October, The (1990)",Action|Thriller


### (9) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [29]:
# user 6041(jyp)이 좋아할만한 영화를 추천
user = 6041
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(1221, 0.5589933),
 (2858, 0.484159),
 (3114, 0.4576686),
 (2028, 0.4403717),
 (589, 0.42966604),
 (2355, 0.37491828),
 (1265, 0.37348032),
 (1580, 0.37167335),
 (2023, 0.29028964),
 (356, 0.26333356),
 (34, 0.2553589),
 (2571, 0.25429612),
 (2997, 0.25282386),
 (1259, 0.23477054),
 (1270, 0.21992731),
 (1721, 0.2169826),
 (457, 0.21163481),
 (2599, 0.20472074),
 (260, 0.20151162),
 (1610, 0.1957546)]

In [30]:
# 좋아할만한 영화와 수치 장르 확인
for i, k in movie_recommended:
    print(movies[movies['movie_id']==i]['title'].values[0],' :',k)

Godfather: Part II, The (1974)  : 0.5589933
American Beauty (1999)  : 0.484159
Toy Story 2 (1999)  : 0.4576686
Saving Private Ryan (1998)  : 0.4403717
Terminator 2: Judgment Day (1991)  : 0.42966604
Bug's Life, A (1998)  : 0.37491828
Groundhog Day (1993)  : 0.37348032
Men in Black (1997)  : 0.37167335
Godfather: Part III, The (1990)  : 0.29028964
Forrest Gump (1994)  : 0.26333356
Babe (1995)  : 0.2553589
Matrix, The (1999)  : 0.25429612
Being John Malkovich (1999)  : 0.25282386
Stand by Me (1986)  : 0.23477054
Back to the Future (1985)  : 0.21992731
Titanic (1997)  : 0.2169826
Fugitive, The (1993)  : 0.21163481
Election (1999)  : 0.20472074
Star Wars: Episode IV - A New Hope (1977)  : 0.20151162
Hunt for Red October, The (1990)  : 0.1957546


In [31]:
#추천영화 장르 확인 
movies[movies['movie_id'].isin([m[0] for m in movie_recommended])][['title','genre']]

,title,genre
33,Babe (1995),Children's|Comedy|Drama
257,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
352,Forrest Gump (1994),Comedy|Romance|War
453,"Fugitive, The (1993)",Action|Thriller
585,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
1203,"Godfather: Part II, The (1974)",Action|Crime|Drama
1239,Stand by Me (1986),Adventure|Comedy|Drama
1245,Groundhog Day (1993),Comedy|Romance
1250,Back to the Future (1985),Comedy|Sci-Fi
1539,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi


# 회고

  - 기분에 따라 바뀔 수도 있는 '선호도'라는 애매모호한 개념을 수치화 해서 추천을 한다는 게 어려운 일인 듯 하지만 비슷한 장르와 좋아할 만한 영화를 대체적으로 잘 뽑아내 주었다. 적어도 내가 '비선호하는' 장르의 영화는 추천되지 않은 것으로 보아 성능은 괜찮은 것 같다. 
<br>   

  - 내가 선호하는 영화로 뽑은 5개의 영화가 장르가 여러가지인데, 그래서 그런지 브레이브하트와는 완전 다른 장르의 터미네이터, Sixth Sense 등이 추천되어 favourite 이 어떤 것이냐는것 보다 앞서 입력한 5개의 영화가 결과에 더 큰 영향을 준것으로 보인다. 앞으로 고잉디퍼와 아이펠톤 등을 거치며 더 적합한 코드, 모델설계를 구현하게 된다면 성능은 더 좋게 할 수 있을거라 믿는다. 
<br>   

  - 금번 과제는 직전 몇개의 과제들보다 무겁지는 않아 많은 에러를 냈음에도 시간에 쫒기지 않고 해결하고 비교적 수월하게 결과를 도출 할 수 있었던 것 같다. 앞으로도 더 다양한 딥러닝 문제들을 시도해 보고 싶다는 생각이 들었다. 
  